# Summer Roadtrip

**Project Description**

Generate a roadtrip route through the most picturesque and popular landmarks and natural attractions. Use available data (e.g., photo frequency, reviews) to rank stops and suggest how long to spend at each.

- Build a route display with a map and scrollable landmark cards.
- Create a visual layout highlighting landmark photos and descriptions.
- Add filtering or sorting by popularity, type (e.g., “nature,” “architecture”), or time required.
- Display total trip summary (e.g., distance, number of stops).

## Find and Prepare Data

Collect data on locations;
- Must contain longitude and latitude
- Location name
- Select a maximum of 9 locations to find the optimal route in a reasonable time
- Some datasets on Kaggle

**Questions**

Is it best to use a Kaggle dataset? Here is one we had in mind: https://www.kaggle.com/datasets/meetnagadia/road-trip-dataset

If we come up with our own, do we need to create features like distances in miles and minutes from landmark to landmark?

## Modeling

First, CREATE BASELINE MODEL
-Create a first route and visualize it
-Select the next location randomly

-Each route point should be visited only once
-Compute the total distance after visiting all locations (it would allow you to evaluate the improvement of the route by your optimization model).

After your baseline model (random) is established, you need to develop an approach to optimize your route. 
- i.e., find the route with the shortest total distance.
- The total distance is the objective function, which should be minimized
- The task to find the optimal distance is a little bit different from usual ML modeling, which we talked about a lot on the platform
- To minimize the distance, you need to come up with an algorithm that selects the next location to visit in such a way that the total distance is minimized (and no location is visited twice).

After your model is created and run, please 
- Calculate the total distance for the optimized route
- Time needed to find this optimal route.
- It would be nice if both your baseline and optimized solutions were visible on the screen for comparison.

## Visualizations

The visualizations should be interactive
	- What elements will be interactive
	- How the end-user should interact with it

Visualizations you can use libraries such as plotly 
- Example: https://plotly.com/python/maps/
- More examples https://python-graph-gallery.com/
  
Visualize 
- Statistics for locations or travel times
- Include a map with locations and the optimal route on this map
- Initial and optimized total distance and time needed for the model to calculate the optimal route.

## Conclusion